In [1]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from utils import read_config, MySQLAgent
import time, random

In [2]:
config = read_config('.env/local_conn.json')
sql_agent = MySQLAgent(config['local_mysql'])

In [3]:
query = """
show tables
"""

sql_agent.read_table(query=query)

,tables_in_generaldb
0,balancesheet
1,port_activity


In [4]:
# 網站連線資料
BalanceSheetURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb03";      # 資產負債表
ProfitAndLoseURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb04";    # 損益表
CashFlowStatementURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb05"; # 現金流量表

In [6]:
def get_balancesheet(stock_number, year, season, url):
    df = pd.DataFrame()
    form_data = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'co_id':stock_number,
        'year': year,
        'season': season,
    }

    try:
        r = requests.post(url,form_data)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find_all('table')[1]
        # headers = [header.text for header in table.find_all('th')]
        year_seaon = table.find_all('th')[0].get_text()
        rows = []
        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            cells_text = [cell.text.strip() for cell in cells]
            rows.append(cells_text)
            
        # the format of season 4 is different with each other
        if season != 4:
            columns = ['acct_name', 'this_year_amt', 'this_year_percent', 'last_hy_amt', 'last_hy_amt_%', 'last_year_amt', 'last_year_percent']
            df_temp = pd.DataFrame(rows, columns=columns)
            df_select = df_temp.iloc[:, [0,1,2,5,6]].iloc[3:].copy()
        elif season == 4:
            columns = ['acct_name', 'this_year_amt', 'this_year_percent', 'last_year_amt', 'last_year_percent']
            df_temp = pd.DataFrame(rows, columns=columns)
            df_select = df_temp.iloc[3:].copy()
        else:
            raise print('wrong season input.')
      
        df_select['report_name'] = 'BalanceSheet'
        df_select['report_time_raw'] = year_seaon
        df_select['period_year'] = year
        df_select['season'] = season
        df_select['creation_date'] = datetime.now().strftime('%Y-%m-%d')
        

        df_select['stock'] = stock_number

        df = pd.concat([df, df_select], axis=0)
        
        
        
        # time.sleep(random.uniform(10, 35))
            
    except requests.RequestException as e:
        print(f'An unexpected error occurred for stock number {stock_number}: {e}')
        
    return df

In [39]:
# stock_number = '2330'
# year = 113
# season = 1
# url = BalanceSheetURL
# df_balancesheet = get_balancesheet(stock_number, year, season, url=url)

get 2330 balancesheet


In [15]:
def balancesheet_crawler(stock_number, url):

    current_year = datetime.now().year - 1911
    year_list = [i for i in range(current_year-5, current_year+1,1)]
    season_list = [1, 2, 3, 4]
    max_year = current_year
    max_season_by_max_year = 1 # hard code
    df_result = pd.DataFrame()
    for year in year_list:
        for season in season_list:
            if year == max_year and season > max_season_by_max_year:
                break
            df_temp = get_balancesheet(stock_number, year, season, url=url)
            print(f'Get {stock_number} balancesheet with year {year}, season {season}')
            
            df_result = pd.concat([df_result, df_temp], axis=0)
                
        
    return df_result

In [16]:
df_balancesheet = balancesheet_crawler(stock_number='2330', url=BalanceSheetURL)

get 2330 balancesheet with year is 108, season is 1
get 2330 balancesheet with year is 108, season is 2
get 2330 balancesheet with year is 108, season is 3
get 2330 balancesheet with year is 108, season is 4
get 2330 balancesheet with year is 109, season is 1
get 2330 balancesheet with year is 109, season is 2
get 2330 balancesheet with year is 109, season is 3
get 2330 balancesheet with year is 109, season is 4
get 2330 balancesheet with year is 110, season is 1
get 2330 balancesheet with year is 110, season is 2
get 2330 balancesheet with year is 110, season is 3
get 2330 balancesheet with year is 110, season is 4
get 2330 balancesheet with year is 111, season is 1
get 2330 balancesheet with year is 111, season is 2
get 2330 balancesheet with year is 111, season is 3
get 2330 balancesheet with year is 111, season is 4
get 2330 balancesheet with year is 112, season is 1
get 2330 balancesheet with year is 112, season is 2
get 2330 balancesheet with year is 112, season is 3
get 2330 bal

IndexError: list index out of range

In [12]:
df_balancesheet

NameError: name 'df_balancesheet' is not defined

In [41]:
sql_agent.write_table(df_balancesheet, 'balancesheet', if_exists='replace', index=False, data_type=None)